In [ ]:
import wasmtime

engine = wasmtime.Engine()
# Enable WASM debug info when creating the engine
config = wasmtime.Config()
config.debug_info = True
config.wasm_backtrace = True
engine = wasmtime.Engine(config)

store = wasmtime.Store(engine)

def log_utf8_caller(caller, ptr, len):
    memory = caller.get_export('memory')
    assert isinstance(memory, wasmtime.Memory)
    data = memory.data_ptr(store)[ptr:ptr + len]
    print("log_utf8 called with:", data.decode('utf-8'))

# Add debug logging for function calls
def debug_wrapper(caller, *args):
    print(f"Function called with args: {args}")
    try:
        result = log_utf8_caller(caller, *args)
        print(f"Function returned: {result}")
        return result
    except Exception as e:
        print(f"Function error: {e}")
        raise

linker = wasmtime.Linker(engine)

log_func = wasmtime.Func(store, 
                        wasmtime.FuncType([wasmtime.ValType.i64(), wasmtime.ValType.i64()], []), 
                        debug_wrapper)  # Wrap the original function

linker.define(
    store=store,
    module="env",
    name="log_utf8",
    item=log_func
)

# Try to catch and print more detailed error info
try:
    wasm_path = "test.wasm"
    with open(wasm_path, "rb") as f:
        module = wasmtime.Module(engine, f.read())
    
    instance = linker.instantiate(store, module)
    print("WASM module instantiated successfully!")
    
    # Print module info
    print("\nModule Exports:")
    for name in instance.exports(store):
        export = instance.exports(store)[name]
        print(f"- {name}: {type(export)}")
    

except Exception as e:
    print(f"Error: {type(e).__name__}: {str(e)}")


# Module Exports:
# - memory: <class 'wasmtime._memory.Memory'>
# - _initialize: <class 'wasmtime._func.Func'>
# - main: <class 'wasmtime._func.Func'>
# - cabi_realloc: <class 'wasmtime._func.Func'>

In [ ]:
# No problem with calling this function
try:
    _initialize = instance.exports(store)['_initialize']
    print("\nCalling _initialize()...")
    result = _initialize(store)
    print(f"_initialize() returned: {result}")
except wasmtime.Trap as e:
    print("\nWASM Trap Error Details:")
    print(f"Message: {str(e)}")
    print(f"Trace:")
    for frame in e.trace():
        print(f"  - {frame}")

In [ ]:
#Throws unreachable
# try:
#     cabi_realloc = instance.exports(store)['cabi_realloc']
#     print("\nCalling cabi_realloc()...")
#     result = cabi_realloc(store,100,100,100,100)
#     print(f"cabi_realloc() returned: {result}")
# except wasmtime.Trap as e:
#     print("\nWASM Trap Error Details:")
#     print(f"Message: {str(e)}")
#     print(f"Trace:")
#     for frame in e.trace():
#         print(f"  - {frame}")

In [ ]:
# Throws unreachable
try:
    helloworld = instance.exports(store)['helloworld']
    print("\nCalling helloworld()...")
    result = helloworld(store)
    print(f"helloworld() returned: {result}")
except wasmtime.Trap as e:
    print("\nWASM Trap Error Details:")
    print(f"Message: {str(e)}")
    print(f"Trace:")
    for frame in e.trace():
        print(f"  - {frame}")